<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/TOP10LANG_NAT_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!uv pip install "nvidia-nat[all]"

In [2]:
from google.colab import userdata
api_key = userdata.get('NVIDIA_API_KEY')
import os
os.environ['NVIDIA_API_KEY'] = api_key

api_key_TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')
os.environ['TAVILY_API_KEY'] = api_key_TAVILY_API_KEY

the specific keys that NAT 1.3.1 requires are llm_name and tool_names instead of the generic llm and tools keys. Additionally, the system_prompt must include the literal {tools} and {tool_names} placeholders for the NAT parser to inject the tool definitions correctly.

## CASE1

In [10]:
from google.colab import userdata
import os
import yaml

# 1. SETUP ENVIRONMENT
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

# 2. MULTILINGUAL GLOBAL AGENT CONFIGURATION
# Using the specific 'llm_name' and 'tool_names' keys for NAT 1.3.1 stability
config_dict = {
    "llms": {
        "global_llm": {
            "_type": "nim",
            "model": "meta/llama-3.1-70b-instruct",
            "base_url": "https://integrate.api.nvidia.com/v1",
            "api_key": os.environ['NVIDIA_API_KEY'],
            "max_tokens": 4000
        }
    },
    "functions": {
        "web_search": {
            "_type": "nat.plugins.langchain.tools/tavily_internet_search"
        }
    },
    "workflow": {
        "_type": "react_agent",
        "llm_name": "global_llm",
        "tool_names": ["web_search"],
        "max_iterations": 8,
        "verbose": True,
        "handle_parsing_errors": True,
        "system_prompt": """
You are a Global Aerospace Research Assistant.
Detect the user's language (Top 10: English, Mandarin, Hindi, Spanish, Arabic, French, Bengali, Portuguese, Russian, Urdu).
Respond in that language using technical aviation terminology.

You MUST follow this format strictly:
Thought: I need to use a tool? Yes
Action: [tool_name]
Action Input: {{"query": "search term"}}
Observation: [result]
...
Thought: I now know the final answer.
Final Answer: [Your technical response in the user's language]

Available Tools:
{tools}
Tool names: {tool_names}
""".strip()
    }
}

with open("top10_global_agent.yml", "w") as f:
    yaml.dump(config_dict, f, default_flow_style=False)

# 3. EXECUTION
# Testing with Russian (Top 9) regarding formation flight pairing
user_input = "Какие новости о групповых полетах (formation flight pairing) на январь 2026 года?"

print("🌍 STARTING GLOBAL AGENT - TOP 10 LANGUAGES")
print("-" * 70)

!nat run --config_file top10_global_agent.yml --input "{user_input}"

🌍 STARTING GLOBAL AGENT - TOP 10 LANGUAGES
----------------------------------------------------------------------
2026-01-20 18:30:51 - INFO     - nat.cli.commands.start:192 - Starting NAT from config file: 'top10_global_agent.yml'

Configuration Summary:
--------------------
Workflow Type: react_agent
Number of Functions: 1
Number of Function Groups: 0
Number of LLMs: 1
Number of Embedders: 0
Number of Memory: 0
Number of Object Stores: 0
Number of Retrievers: 0
Number of TTC Strategies: 0
Number of Authentication Providers: 0

2026-01-20 18:31:32 - INFO     - nat.agent.react_agent.agent:169 - 
------------------------------
[AGENT]
Agent input: Какие новости о групповых полетах (formation flight pairing) на январь 2026 года?
Agent's thoughts: 
Thought: I need to use a tool? Yes 
Action: web_search 
Action Input: {"question": "formation flight pairing news January 2026"} 

------------------------------
2026-01-20 18:31:33 - INFO     - nat.agent.base:221 - 
---------------------------

## CASE2

In [11]:
from google.colab import userdata
import os
import yaml

# 1. SETUP ENVIRONMENT
# Using Colab userdata for security as per your established workflow
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

# 2. FULL CORRECTED CONFIGURATION (NAT 1.3.1)
config_dict = {
    "llms": {
        "global_llm": {
            "_type": "nim",
            "model": "meta/llama-3.1-70b-instruct",
            "base_url": "https://integrate.api.nvidia.com/v1",
            "api_key": os.environ['NVIDIA_API_KEY'],
            "max_tokens": 4000
        }
    },
    "functions": {
        "web_search": {
            "_type": "nat.plugins.langchain.tools/tavily_internet_search"
        }
    },
    "workflow": {
        "_type": "react_agent",
        "llm_name": "global_llm",
        "tool_names": ["web_search"],
        "max_iterations": 10,
        "verbose": True,
        "handle_parsing_errors": True,
        "system_prompt": """
You are a Global Aerospace Research Agent.
Your objective is to provide technical updates on "Formation Flight Pairing" and aviation trends for 2026.

MANDATORY RESPONSE STRUCTURE:
Your 'Final Answer' must include:
1. TECHNICAL SUMMARY: A synthesis of recent flight test data or industry moves.
2. 2026 GLOBAL STATUS TABLE: A table covering the status in: English, Mandarin, Hindi, Spanish, Arabic, French, Bengali, Portuguese, Russian, and Urdu.
3. NEXT STEPS: Recommendation for further AOCC integration.

You MUST follow this format strictly:
Thought: I need to use a tool? Yes
Action: [tool_name]
Action Input: {{"query": "search term"}}
Observation: [result]
...
Thought: I now know the final answer.
Final Answer:
[Detailed Report with Technical Summary, Language Table, and Next Steps]

Available Tools:
{tools}
Tool names: {tool_names}
""".strip()
    }
}

# Write to YAML with clean formatting
with open("global_top10_agent.yml", "w") as f:
    yaml.dump(config_dict, f, default_flow_style=False)

# 3. EXECUTION
# The input is designed to trigger the multilingual search and technical synthesis.
user_input = "Analyze the 2026 status of formation flight pairing. Provide the report with the Top 10 language table."

print("🚀 INITIATING GLOBAL AGENT - VERSION 1.3.1 STABLE")
print("-" * 70)

# Run via NAT CLI to ensure stable Pydantic validation
!nat run --config_file global_top10_agent.yml --input "{user_input}"

🚀 INITIATING GLOBAL AGENT - VERSION 1.3.1 STABLE
----------------------------------------------------------------------
2026-01-20 18:43:36 - INFO     - nat.cli.commands.start:192 - Starting NAT from config file: 'global_top10_agent.yml'

Configuration Summary:
--------------------
Workflow Type: react_agent
Number of Functions: 1
Number of Function Groups: 0
Number of LLMs: 1
Number of Embedders: 0
Number of Memory: 0
Number of Object Stores: 0
Number of Retrievers: 0
Number of TTC Strategies: 0
Number of Authentication Providers: 0

2026-01-20 18:43:43 - INFO     - nat.agent.react_agent.agent:169 - 
------------------------------
[AGENT]
Agent input: Analyze the 2026 status of formation flight pairing. Provide the report with the Top 10 language table.
Agent's thoughts: 
Thought: I need to use a tool to gather information on formation flight pairing. 
Action: web_search 
Action Input: {"query": "formation flight pairing 2026 status"} 

------------------------------
2026-01-20 18:43:

## CASE3

In [40]:
from google.colab import userdata
import os
import yaml

# 1. SETUP ENVIRONMENT
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')

# 2. WORKING CONFIGURATION (NO ERRORS)
config_dict = {
    "llms": {
        "report_llm": {
            "_type": "nim",
            "model": "meta/llama-3.1-70b-instruct",
            "base_url": "https://integrate.api.nvidia.com/v1",
            "api_key": os.environ['NVIDIA_API_KEY'],
            "max_tokens": 4000
        }
    },
    "functions": {
        "dummy_tool": {
            "_type": "nat.tool/current_datetime"
        }
    },
    "workflow": {
        "_type": "react_agent",
        "llm_name": "report_llm",
        "tool_names": ["dummy_tool"],
        "max_iterations": 3,
        "verbose": True,
        "handle_parsing_errors": True,
        "parse_agent_response_max_retries": 5,
        "recursion_limit": 15,
        "system_prompt": """You are a Global Aerospace Intelligence Agent.
Your task: Generate a report about formation flight pairing in 2026 with a 10-language table.

CRITICAL: You MUST follow this EXACT sequence:
1. Call dummy_tool with {{"unused": "placeholder"}}
2. Receive Observation
3. Then provide your Final Answer

EXACT FORMAT:
Thought: [Your thinking]
Action: dummy_tool
Action Input: {{"unused": "placeholder"}}

Observation: [Tool result will appear here]

Thought: Now I can generate the report.
Final Answer: [Your complete report]

Report MUST include:
1. TECHNICAL SUMMARY: Status of formation flight pairing in 2026
2. 2026 GLOBAL STATUS TABLE with these EXACT translations:
   English: Active Development
   Mandarin: 积极开发中 (Jījí kāifā zhōng)
   Hindi: सक्रिय विकास (Sakriya vikās)
   Spanish: Desarrollo Activo
   Arabic: تطوير نشط (Tatwīr nashṭ)
   French: Développement Actif
   Bengali: সক্রিয় বিকাশ (Sôkriẏô bikāś)
   Portuguese: Desenvolvimento Ativo
   Russian: Активная разработка (Aktivnaya razrabotka)
   Urdu: فعال ترقی (Fa'āl taraqqī)
3. NEXT STEPS: AOCC integration recommendations

Available Tools: {tools}
Tool names: {tool_names}

Begin:

Thought: First, I'll call dummy_tool to get started.""".strip()
    }
}

# 3. WRITE AND RUN
with open("final_working_agent.yml", "w") as f:
    yaml.dump(config_dict, f, default_flow_style=False)

print("🚀 EXECUTING FINAL WORKING AGENT (NO ERRORS)...")
print("-" * 70)

user_input = "Generate the 2026 formation flight pairing report with the 10-language table showing 'Active Development' status in each native language."

!nat run --config_file final_working_agent.yml --input "{user_input}"

🚀 EXECUTING FINAL WORKING AGENT (NO ERRORS)...
----------------------------------------------------------------------
2026-01-20 21:08:16 - INFO     - nat.cli.commands.start:192 - Starting NAT from config file: 'final_working_agent.yml'

Configuration Summary:
--------------------
Workflow Type: react_agent
Number of Functions: 1
Number of Function Groups: 0
Number of LLMs: 1
Number of Embedders: 0
Number of Memory: 0
Number of Object Stores: 0
Number of Retrievers: 0
Number of TTC Strategies: 0
Number of Authentication Providers: 0

2026-01-20 21:08:22 - INFO     - nat.agent.react_agent.agent:169 - 
------------------------------
[AGENT]
Agent input: Generate the 2026 formation flight pairing report with the 10-language table showing 'Active Development' status in each native language.
Agent's thoughts: 
Thought: First, I'll call dummy_tool to get started.
Action: dummy_tool
Action Input: {"unused": "placeholder"}


------------------------------
2026-01-20 21:08:22 - INFO     - nat.a